In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import re

In [5]:
url = 'https://puzzles-prod.telegraph.co.uk/plusword/data.json'
r_json = requests.get(url).json()

row = {'date' : r_json['copy']['date-publish-analytics'][:10],
      'puzzle_number' : r_json['meta']['number'],
      'id' :r_json['meta']['id'],
      'plusword_solution' : r_json['settings']['solution']}

for direction in ['across', 'down']:
    clue_num = 0
    for clue in r_json['cluedata'][direction]:
        clue_num +=1
        row.update({'clue_' + direction + '_' + str(clue_num) :clue})

for answer_num in range(1,6):
    row.update({'answer_' + str(answer_num) :r_json['celldata'][5*(answer_num-1):(5*answer_num)]})

In [6]:
url = 'https://puzzles-prod.telegraph.co.uk/plusword/index.html'
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
driver.get(url)
element = driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/header/button').click()

yellow = []
green = []
match = 'C\d*'
for table_row in driver.find_elements(By.CLASS_NAME, "row"):
    for cell in table_row.find_elements(By.TAG_NAME, 'td'):
        cell_class = cell.get_attribute("class")
                
        if 'right-letter-wrong-column' in cell_class:
            yellow.append((re.search(match, cell_class).group(0).strip('C')))
            
        if 'right-letter-right-column' in cell_class:
            green.append((re.search(match, cell_class).group(0).strip('C')))
            
row.update({'yellow' : yellow,
            'green' : green})

In [7]:
df = pd.DataFrame.from_records([row])


df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

df

,date,puzzle_number,id,plusword_solution,clue_across_1,clue_across_2,clue_across_3,clue_across_4,clue_across_5,clue_down_1,...,clue_down_3,clue_down_4,clue_down_5,answer_1,answer_2,answer_3,answer_4,answer_5,yellow,green
0,2023-04-22,335,25564,PAINT,Sends junk email to,Jewish scripture,Cream of the crop,Let up,Garb,Mount such as a horse,...,Crop up,Makes the winning move at chess,Outbuildings,SPAMS,TORAH,ELITE,EASED,DRESS,"[1, 2, 5, 8, 13]","[12, 16]"


In [8]:
df.dtypes

date                 datetime64[ns]
puzzle_number                object
id                           object
plusword_solution            object
clue_across_1                object
clue_across_2                object
clue_across_3                object
clue_across_4                object
clue_across_5                object
clue_down_1                  object
clue_down_2                  object
clue_down_3                  object
clue_down_4                  object
clue_down_5                  object
answer_1                     object
answer_2                     object
answer_3                     object
answer_4                     object
answer_5                     object
yellow                       object
green                        object
dtype: object